In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 704.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 64.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2

In [6]:
import gradio as gr
import tensorflow as tf
import numpy as np
import cv2

# Tải mô hình
model = tf.keras.models.load_model('VGG16_model.keras')

img_height, img_width = 224, 224

# Tên các loài hoa và thông tin chi tiết
class_names = ['Cúc', 'Bồ Công Anh', 'Hoa Hồng', 'Hướng Dương', 'Tulip']

flower_info = {
    'Cúc': {
        'Nguồn gốc': 'Cúc có nguồn gốc từ châu Âu và châu Á.',
        'Đặc điểm nhận dạng': 'Hoa nhỏ, màu trắng hoặc vàng, có tâm màu vàng.',
        'Kích thước': 'Chiều cao khoảng 20-30 cm.',
        'Thông tin thêm': 'Cúc biểu tượng cho sự trong sáng và thuần khiết.'
    },
    'Bồ Công Anh': {
        'Nguồn gốc': 'Bồ Công Anh phân bố rộng khắp vùng ôn đới châu Âu và Bắc Mỹ.',
        'Đặc điểm nhận dạng': 'Hoa nhỏ, màu vàng, nở thành cụm tròn.',
        'Kích thước': 'Chiều cao khoảng 10-30 cm.',
        'Thông tin thêm': 'Hạt bồ công anh được phát tán nhờ gió, biểu tượng cho tự do.'
    },
    'Hoa Hồng': {
        'Nguồn gốc': 'Hoa hồng có nguồn gốc từ châu Á.',
        'Đặc điểm nhận dạng': 'Cánh hoa dày, màu sắc đa dạng, thường có gai trên thân.',
        'Kích thước': 'Chiều cao từ 50-200 cm.',
        'Thông tin thêm': 'Hoa hồng là biểu tượng của tình yêu và sự lãng mạn.'
    },
    'Hướng Dương': {
        'Nguồn gốc': 'Hướng dương có nguồn gốc từ Bắc Mỹ.',
        'Đặc điểm nhận dạng': 'Hoa lớn, màu vàng tươi, có tâm màu nâu.',
        'Kích thước': 'Chiều cao từ 1-3 mét.',
        'Thông tin thêm': 'Hoa hướng dương biểu tượng cho sự lạc quan và hướng tới ánh sáng.'
    },
    'Tulip': {
        'Nguồn gốc': 'Tulip có nguồn gốc từ Trung Đông và châu Âu.',
        'Đặc điểm nhận dạng': 'Cánh hoa hình tròn hoặc oval, nhiều màu sắc rực rỡ.',
        'Kích thước': 'Chiều cao từ 10-70 cm.',
        'Thông tin thêm': 'Tulip là biểu tượng của sự thịnh vượng và tình yêu viên mãn.'
    }
}

def predict_image(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

    # Xử lí hình ảnh
    img = cv2.resize(image, (img_height, img_width))
    img = np.array(img).reshape(-1, img_height, img_width, 3)
    img = img / 255.0

    # Dự đoán
    predictions = model.predict(img)

    # Lấy nhãn dự đoán
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    predicted_class_name = class_names[predicted_class_index]

    # Lấy thông tin chi tiết về loài hoa
    flower_details = flower_info.get(predicted_class_name, "Không có thông tin")

    # Trả về tên hoa và thông tin chi tiết
    result = f"Tên hoa: {predicted_class_name}\n\n"
    for key, value in flower_details.items():
        result += f"{key}: {value}\n"

    return result

# Tạo Gradio interface
iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Phân loại hoa bằng VGG16",
    description="Tải lên hình ảnh để phân loại và nhận thông tin về loài hoa."
)

# Chạy app
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7e8ba5e3214b7445a5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
